In [1]:
import argparse
import json
import os
import pprint
import sys
from pathlib import Path

import cv2
import h5py
# import hydra
import numpy as np
import torch
import torchvision
import yaml
from easydict import EasyDict
# from hydra.experimental import compose, initialize
# from omegaconf import DictConfig, OmegaConf
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from deoxys import config_root
# Control import
from deoxys.franka_interface import FrankaInterface
# from deoxys.utils import RobotStateRawObsDictGenerator, YamlConfig
from deoxys.utils import  YamlConfig
from deoxys.utils.config_utils import robot_config_parse_args
from deoxys.utils.input_utils import input2action
from deoxys.utils.io_devices import SpaceMouse
from deoxys.utils.log_utils import get_deoxys_example_logger
from deoxys.experimental.motion_utils import follow_joint_traj, reset_joints_to


import argparse
import json
import numpy as np
import time
import os
import shutil
import psutil
import sys
import socket
import traceback

from collections import OrderedDict

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
from robomimic.utils.log_utils import PrintLogger, DataLogger, flush_warnings


import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.tensor_utils as TensorUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils

import imageio 

import matplotlib.pyplot as plt

from robomimic.envs.wrappers import EnvWrapper
from copy import deepcopy
import textwrap
import numpy as np
from collections import deque
import time 
# from camera_utils import MyRealSense, CVCamera

np.set_printoptions(precision=3, suppress=True)

logger = get_deoxys_example_logger()

pybullet build time: Nov 28 2023 23:45:17
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/franka_deoxys/robomimic/robomimic/scripts/setup_macros.py
)


/home/franka_deoxys/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)
device 

device(type='cuda', index=0)

In [3]:
class BaseRawObsDictGenerator:
    def __init__(self, *args, **kwargs):
        self.last_obs_dict = None

    def get_raw_obs_dict(self, state_info):
        """
        Args:
           state_info (dict): A dictionary of robot state + images
        """
        obs_dict = {}
        raise NotImplementedError

    def load(self):
        raise NotImplementedError


class RobotStateRawObsDictGenerator(BaseRawObsDictGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def load(self, obs_dict, key, value, check_valid=True):
        """
        This is to check if the data is correct or not. Sometimes the data will be all zero depending on the networking conditions.
        """
        if (
            (
                np.sum(np.abs(value)) == 0.0
                and key in ["ee_states", "joint_states", "gripper_states"]
            )
            and check_valid
            and self.last_obs_dict is not None
        ):
            value = self.last_obs_dict[key]
        obs_dict[key] = value

    def get_raw_obs_dict(self, state_info):
        last_state = state_info["last_state"]
        last_gripper_state = state_info["last_gripper_state"]
        obs_dict = {}

        ee_states = np.array(last_state.O_T_EE)
        joint_states = np.array(last_state.q)
        gripper_states = np.array([last_gripper_state.width])

        self.load(obs_dict, "ee_states", ee_states)
        self.load(obs_dict, "joint_states", joint_states)
        # Gripper widh will probably become zero
        self.load(obs_dict, "gripper_states", gripper_states, check_valid=False)

        for state in ["ee_states", "joint_states", "gripper_states"]:
            if (
                np.sum(np.abs(obs_dict[state])) <= 1e-6
                and self.last_obs_dict is not None
            ):
                print(f"{state} missing!!!!")
                obs_dict[state] = self.last_obs_dict[state]
        self.last_obs_dict = obs_dict
        return obs_dict

In [4]:
def stacked_get_init(init_obs, num_frames):
    obs_history = {}
    for k in init_obs:
        obs_history[k] = deque([init_obs[k][None] for _ in range(num_frames)], maxlen=num_frames,)
    obs = { k : np.concatenate(obs_history[k], axis=0) for k in obs_history }
    return obs_history, obs 

def stacked_add_new(obs_history, new_obs):
    for k in new_obs:
        if 'timesteps' in k or 'actions' in k: continue
        obs_history[k].append(new_obs[k][None])

    obs= { k : np.concatenate(obs_history[k], axis=0) for k in obs_history }
    return obs_history, obs 

In [5]:
# dataset_path="/home/franka_deoxys/data_franka/block_green_on_blue/franka_block_gonb_50.hdf5"
# ckpt_path = "/home/franka_deoxys/data_franka/block_green_on_blue/policy_low/model_epoch_120.pth"
# assert os.path.exists(ckpt_path)

In [6]:
dataset_path="/home/franka_deoxys/data_franka/block_green_on_blue/franka_gonb50.hdf5"
ckpt_path = "/home/franka_deoxys/data_franka/block_green_on_blue/policy_low/trans_epoch_60.pth"
assert os.path.exists(ckpt_path)

In [7]:
# restore policy
policy, ckpt_dict = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path, device=device, verbose=False)
ckpt_dict = FileUtils.maybe_dict_from_checkpoint(ckpt_path=ckpt_path, ckpt_dict=None)
# algo name and config from model dict
algo_name, _ = FileUtils.algo_name_from_checkpoint(ckpt_dict=ckpt_dict)
config, _ = FileUtils.config_from_checkpoint(algo_name=algo_name, ckpt_dict=ckpt_dict, verbose=False)

config.unlock()
config.train.data=dataset_path
config.lock() 


ObsUtils.initialize_obs_utils_with_config(config)

shape_meta = FileUtils.get_shape_metadata_from_dataset(
    dataset_path=config.train.data,
    all_obs_keys=config.all_obs_keys,
    verbose=False
)
trainset, validset = TrainUtils.load_data_for_training(
    config, obs_keys=shape_meta["all_obs_keys"])
len(trainset.demos)

/home/franka_deoxys/robomimic/robomimic/utils/file_utils.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt_dict = torch.load(ckpt_path)



============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['gripper_states', 'ee_states', 'joint_states']
using obs modality: rgb with keys: []
using obs modality: depth with keys: []
using obs modality: scan with keys: []
Created GPT_Backbone model with number of parameters: 18906112

============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['gripper_states', 'ee_states', 'joint_states']
using obs modality: rgb with keys: []
using obs modality: depth with keys: []
using obs modality: scan with keys: []
SequenceDataset: loading dataset into memory...
100%|██████████| 50/50 [00:00<00:00, 1353.21it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 3280/3280 [00:00<00:00, 22225.86it/s]


50

In [8]:
model = policy.policy
model.set_eval()

In [9]:
logger.warn("This is a very dummy control policy!!!!!")

args = robot_config_parse_args()
robot_interface = FrankaInterface(os.path.join(config_root, args.interface_cfg))
controller_cfg = YamlConfig(
    os.path.join(config_root, args.controller_cfg)
).as_easydict()
controller_type = args.controller_type

spacemouse = SpaceMouse(vendor_id=9583, product_id=50734)
spacemouse.start_control()

raw_obs_dict_generator = RobotStateRawObsDictGenerator()

import time

time.sleep(0.3)
dummy_torch_model = torch.nn.Linear(7, 1)

[Deoxys Examples WARNING] This is a very dummy control policy!!!!! (Ln 1)


/tmp/ipykernel_105536/2597759035.py:1: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("This is a very dummy control policy!!!!!")


Opening SpaceMouse device
Manufacturer: 3Dconnexion
Product: SpaceMouse Wireless

Control                       	Command
Right button                  	reset simulation
Left button (hold)            	close gripper
Move mouse laterally          	move arm horizontally in x-y plane
Move mouse vertically         	move arm vertically
Twist mouse about an axis     	rotate arm about a corresponding axis
ESC                           	quit



In [10]:
args 

Namespace(interface_cfg='charmander.yml', folder=PosixPath('/home/franka_deoxys/.local/share/jupyter/runtime/kernel-v2-12320ZTwlhMLLGbMN.json'), controller_type='OSC_POSE', controller_cfg='osc-position-controller.yml')

In [11]:
file=trainset.hdf5_file
demo = file['data']['demo_5']
actions=demo['actions']
actions.shape

(58, 7)

In [12]:
# file["data"].attrs["config"]
for key in file["data"].attrs.keys():
    print(key, file["data"].attrs[key])

camera_ids [0 1]
num_demos 50
total 3280


In [13]:
joint_sequence = file["data/demo_1/obs/joint_states"]
joint_sequence.shape

(66, 7)

In [14]:
def set_gripper(open=True):
    d=-1. if open else 1.0
    action_close=np.array([ 0.,  0., -0.,  0.,  0., -0., d])
    robot_interface.control(
        controller_type=controller_type,
        action=action_close,
        controller_cfg=controller_cfg,
    )

In [23]:
logger.info("Resetting to the initial configuration")
reset_joints_to(robot_interface, joint_sequence[0])
set_gripper(open=True)

[Deoxys Examples INFO] Resetting to the initial configuration (Ln 1)
JOINT_POSITION


In [16]:
# set_gripper(open=False)

In [17]:
# for action in actions:
#     robot_interface.control(
#         controller_type=controller_type,
#         action=action,
#         controller_cfg=controller_cfg,
#     )
# robot_interface.close()

In [18]:
def get_current_obs_lowdim(robot_interface):
    last_state = robot_interface._state_buffer[-1]
    last_gripper_state = robot_interface._gripper_state_buffer[-1]

    obs_dict = raw_obs_dict_generator.get_raw_obs_dict(
    {"last_state": last_state, "last_gripper_state": last_gripper_state}) 
    return obs_dict

In [19]:
obs_dict=get_current_obs_lowdim(robot_interface)
for key in obs_dict.keys():
    obs_dict[key]=obs_dict[key][None]
    print(key, obs_dict[key].shape)

ee_states (1, 16)
joint_states (1, 7)
gripper_states (1, 1)


In [20]:
num_frames=10

In [21]:
init_obs = get_current_obs_lowdim(robot_interface)
obs_history, obs = stacked_get_init(init_obs, num_frames) 
obs=TensorUtils.to_torch(obs, device=device)

batch_obs={key:obs[key][None] for key in obs} 
pred = model.get_action(obs_dict=batch_obs , goal_dict=None)
ac=pred.detach().cpu().numpy().squeeze()
ac

array([ 0.126, -0.02 , -0.306, -0.003, -0.005, -0.005, -1.   ],
      dtype=float32)

In [22]:
actions = []
for _ in range(200):
    action, grasp = input2action(
        device=spacemouse,
        controller_type=controller_type,
    )
    if action is None:
        break
    if len(robot_interface._state_buffer) == 0:
        continue

    obs_t = get_current_obs_lowdim(robot_interface)
    obs_history, obs = stacked_add_new(obs_history, obs_t) 
    obs=TensorUtils.to_torch(obs, device=device)
    batch_obs={key:obs[key][None] for key in obs} 
    pred = model.get_action(obs_dict=batch_obs , goal_dict=None)
    action=pred.detach().cpu().numpy().squeeze() 
    # action=action[:6] 

    actions.append(action)
    robot_interface.control(
        controller_type=controller_type,
        action=action,
        controller_cfg=controller_cfg,
    )

    dt=0.01
    time.sleep(dt)



KeyboardInterrupt: 

### Worked great.

In [23]:
robot_interface.close()

In [25]:
len(actions)

56

In [27]:
actions[:10]
for ac in actions[:10]:
    print(ac)

[ 0.126 -0.02  -0.306 -0.003 -0.005 -0.005 -1.   ]
[ 0.126 -0.021 -0.306 -0.003 -0.006 -0.005 -1.   ]
[ 0.127 -0.02  -0.306 -0.003 -0.005 -0.005 -1.   ]
[ 0.136 -0.02  -0.31  -0.003 -0.006 -0.005 -1.   ]
[ 0.149 -0.019 -0.314 -0.003 -0.006 -0.005 -1.   ]
[ 0.163 -0.018 -0.319 -0.003 -0.006 -0.005 -1.   ]
[ 0.181 -0.018 -0.326 -0.003 -0.006 -0.005 -1.   ]
[ 0.2   -0.017 -0.333 -0.003 -0.006 -0.005 -1.   ]
[ 0.222 -0.016 -0.342 -0.003 -0.006 -0.005 -1.   ]
[ 0.241 -0.015 -0.349 -0.003 -0.006 -0.005 -1.   ]
